In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import argparse, csv, sys, collections

df = pd.DataFrame(pd.read_table('round1_ijcai_18_train_20180301.txt', header=0, delim_whitespace=True))

'''----------------将编号重新labelEncoder-------------------'''
le = preprocessing.LabelEncoder()
#item_id 数据转换
le.fit(df['item_id'])
df['item_id']=le.transform(df['item_id'])
#item_brand_id 数据转换
le.fit(df['item_brand_id'])
df['item_brand_id']=le.transform(df['item_brand_id'])
#item_city_id 数据转换
le.fit(df['item_city_id'])
df['item_city_id']=le.transform(df['item_city_id'])
#user_id 数据转换
le.fit(df['user_id'])
df['user_id']=le.transform(df['user_id'])
#context_id 数据转换
le.fit(df['context_id'])
df['context_id']=le.transform(df['context_id'])
#shop_id 数据转换
le.fit(df['shop_id'])
df['shop_id']=le.transform(df['shop_id'])
#instance_id 数据转换
le.fit(df['instance_id'])
df['instance_id']=le.transform(df['instance_id'])

'''
# 时间特征与点击率可视化,时间粒度为小时(可调),count里存放单位时间(小时)内点击数量的平均值,画出count和time的折线图
df['context_timestamp'] = pd.to_datetime(df['context_timestamp'], unit='s')
df['context_timestamp'].hist(bins=100)
df['time'] = df['context_timestamp'].dt.strftime('%Y-%m-%d-%H')

def mean_time(group):
    group['count'] = group['is_trade'].mean()
    return group

df = df.groupby('time').apply(mean_time)
df.plot(x='time', y='count', kind='line')
#plt.show()
'''

df['item_sales_level']=df['item_sales_level'].replace(-1, np.nan)
df['item_sales_level']=df['item_sales_level'].replace(np.nan, df['item_sales_level'].mean()) #设为均值

#类别数据，直接将-1值置为众数
df['user_gender_id']=df['user_gender_id'].replace(-1, df['user_gender_id'].mode()[0])
df['user_age_level']=df['user_age_level'].replace(-1, df['user_age_level'].mode()[0])
df['user_occupation_id']=df['user_occupation_id'].replace(-1, df['user_occupation_id'].mode()[0])
df['user_star_level']=df['user_star_level'].replace(-1, df['user_star_level'].mode()[0])


#先将-1置为空，然后求得各个值的均值
df['shop_review_positive_rate']=df['shop_review_positive_rate'].replace(-1, np.nan)
df['shop_score_service']=df['shop_score_service'].replace(-1, np.nan)
df['shop_score_delivery']=df['shop_score_delivery'].replace(-1, np.nan)
df['shop_score_description']=df['shop_score_description'].replace(-1, np.nan)

# 再将空值置为均值
df['shop_review_positive_rate']=df['shop_review_positive_rate'].replace(np.nan, df['shop_review_positive_rate'].mean())
df['shop_score_service']=df['shop_score_service'].replace(np.nan, df['shop_score_service'].mean())
df['shop_score_delivery']=df['shop_score_delivery'].replace(np.nan, df['shop_score_delivery'].mean())
df['shop_score_description']=df['shop_score_description'].replace(np.nan, df['shop_score_description'].mean())


#连续数据处理
df['user_age_level'] = df['user_age_level'].map(lambda x : x-1000)
df['user_star_level'] = df['user_star_level'].map(lambda x : x-3000)
df['context_page_id'] = df['context_page_id'].map(lambda x : x-4000)
df['shop_star_level'] = df['shop_star_level'].map(lambda x : x-4999)

#修改连续数据的列名
df.rename(columns={'item_price_level':'I1', 'item_sales_level':'I2','item_collected_level':'I3','item_pv_level':'I4','user_age_level':'I5',
                   'user_star_level':'I6','context_page_id':'I7','shop_review_num_level':'I8','shop_review_positive_rate':'I9','shop_star_level':'I10',
                   'shop_score_service':'I11','shop_score_delivery':'I12','shop_score_description':'I13',}, inplace = True)

#修改类别变量的列名
df.rename(columns={'item_id':'C1','item_category_list':'C2','item_brand_id':'C3','item_city_id':'C4',
                  'user_id':'C5','user_gender_id':'C6','user_occupation_id':'C7','shop_id':'C8','item_property_list':'C9','predict_category_property':'C10'}, inplace = True)

#修改is_trade列名为Label
df.rename(columns={'is_trade':'Label'}, inplace = True)

#item_category_list列 数据处理
'''
item_category_1 所有的的样本都是同一个值
item_category_2 一共有13个，我选N=10000,这样是取5个（N=5000的话,是取7个）
item_category_3 只有2个值，，而且出现频率太低，才几百，相对2的都太低了，不要了
'''
df['item_category_1'], df['item_category_2'], df['item_category_3'] = df['C2'].str.split(';', -1).str
df['C2'] = df['item_category_2']

#new_columns = ['Label','I1','I2','I3','I4','I5','I6','I7','I8','I9','I10','I11','I12','I13','C1','C2','C3','C4','C5','C6','C7','C8']
#df.to_csv("tr.csv",index=False,columns=new_columns)
#df.to_csv("te.csv",index=False,columns=new_columns)





In [2]:
df.iloc[:15]

,instance_id,C1,C2,C9,C3,C4,I1,I2,I3,I4,...,I8,I9,I10,I11,I12,I13,Label,item_category_1,item_category_2,item_category_3
0,5703,3760,5799347067982556520,2072967855524022579;5131280576272319091;263639...,448,50,3,3.0,4,14,...,4,1.000000,3,1.000000,1.000000,1.000000,0,7908382889764677758,5799347067982556520,NaN
1,298346,3760,5799347067982556520,2072967855524022579;5131280576272319091;263639...,448,50,3,3.0,4,14,...,4,1.000000,3,1.000000,1.000000,1.000000,0,7908382889764677758,5799347067982556520,NaN
2,43785,3760,5799347067982556520,2072967855524022579;5131280576272319091;263639...,448,50,3,3.0,4,14,...,4,1.000000,3,1.000000,1.000000,1.000000,0,7908382889764677758,5799347067982556520,NaN
3,48752,3760,5799347067982556520,2072967855524022579;5131280576272319091;263639...,448,50,3,3.0,4,14,...,4,1.000000,3,1.000000,1.000000,1.000000,0,7908382889764677758,5799347067982556520,NaN
4,413252,3760,5799347067982556520,2072967855524022579;5131280576272319091;263639...,448,50,3,3.0,4,14,...,4,1.000000,3,1.000000,1.000000,1.000000,0,7908382889764677758,5799347067982556520,NaN
5,402258,3760,5799347067982556520,2072967855524022579;5131280576272319091;263639...,448,50,3,3.0,4,14,...,4,1.000000,3,1.000000,1.000000,1.000000,0,7908382889764677758,5799347067982556520,NaN
6,360385,333,8277336076276184272,2072967855524022579;5131280576272319091;263639...,2011,9,8,9.0,8,13,...,15,0.985427,13,0.974878,0.976863,0.969278,0,7908382889764677758,8277336076276184272,NaN
7,434428,333,8277336076276184272,2072967855524022579;5131280576272319091;263639...,2011,9,8,9.0,8,13,...,15,0.985427,13,0.974878,0.976863,0.969278,0,7908382889764677758,8277336076276184272,NaN
8,208286,5703,5755694407684602296,2072967855524022579;5131280576272319091;263639...,1257,9,8,9.0,10,16,...,15,0.985427,13,0.974878,0.976863,0.969278,0,7908382889764677758,5755694407684602296,NaN
9,336887,333,8277336076276184272,2072967855524022579;5131280576272319091;263639...,2011,9,8,9.0,8,13,...,15,0.985427,13,0.974878,0.976863,0.969278,0,7908382889764677758,8277336076276184272,NaN


In [8]:
new_columns = ['Label','I1','I2','I3','I4','I5','I6','I7','I8','I9','I10','I11','I12','I13','C1','C2','C3','C4','C5','C6','C7','C8']
df_new = df[new_columns]

In [12]:
df_new.iloc[:2000]

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8
0,0,3,3.0,4,14,3,3,6,4,1.000000,...,1.000000,1.000000,3760,5799347067982556520,448,50,96499,1,2005,2872
1,0,3,3.0,4,14,2,6,1,4,1.000000,...,1.000000,1.000000,3760,5799347067982556520,448,50,57700,0,2005,2872
2,0,3,3.0,4,14,3,4,1,4,1.000000,...,1.000000,1.000000,3760,5799347067982556520,448,50,112335,0,2005,2872
3,0,3,3.0,4,14,4,6,16,4,1.000000,...,1.000000,1.000000,3760,5799347067982556520,448,50,57475,1,2005,2872
4,0,3,3.0,4,14,2,1,1,4,1.000000,...,1.000000,1.000000,3760,5799347067982556520,448,50,58526,0,2005,2872
5,0,3,3.0,4,14,4,2,3,4,1.000000,...,1.000000,1.000000,3760,5799347067982556520,448,50,96645,1,2005,2872
6,0,8,9.0,8,13,6,0,1,15,0.985427,...,0.976863,0.969278,333,8277336076276184272,2011,9,188938,1,2005,2090
7,0,8,9.0,8,13,2,2,1,15,0.985427,...,0.976863,0.969278,333,8277336076276184272,2011,9,139490,0,2002,2090
8,0,8,9.0,10,16,3,7,1,15,0.985427,...,0.976863,0.969278,5703,5755694407684602296,1257,9,133008,0,2002,2090
9,0,8,9.0,8,13,3,6,3,15,0.985427,...,0.976863,0.969278,333,8277336076276184272,2011,9,129592,0,2005,2090


In [35]:
for row in df_new.loc[:]:
    print(row)

Label
I1
I2
I3
I4
I5
I6
I7
I8
I9
I10
I11
I12
I13
C1
C2
C3
C4
C5
C6
C7
C8


In [20]:
counts = collections.defaultdict(lambda : [0, 0, 0])        
for i in range(0, 20000):
    row = df_new.loc[i]
    label = row['Label']
    for j in range(1, 9):
        field = 'C{0}'.format(j)
        value = row[field]
        if label == 0:
            counts[field+','+str(value)][0] += 1
        else:
            counts[field+','+str(value)][1] += 1
        counts[field+','+str(value)][2] += 1

print('Field,Value,Neg,Pos,Total,Ratio')
for key, (neg, pos, total) in sorted(counts.items(), key=lambda x: x[1][2]):
    if total < 10:
        continue
    ratio = round(float(pos)/total, 5)
    print(key+','+str(neg)+','+str(pos)+','+str(total)+','+str(ratio))

Field,Value,Neg,Pos,Total,Ratio
C1,9128,10,0,10,0.0
C1,308,10,0,10,0.0
C1,3649,10,0,10,0.0
C1,7358,9,1,10,0.1
C1,717,10,0,10,0.0
C3,1041,9,1,10,0.1
C1,295,9,1,10,0.1
C8,1587,10,0,10,0.0
C5,44107,10,0,10,0.0
C5,148759,10,0,10,0.0
C1,407,10,0,10,0.0
C5,194751,11,0,11,0.0
C1,5263,11,0,11,0.0
C1,5735,11,0,11,0.0
C1,7405,11,0,11,0.0
C1,1053,11,0,11,0.0
C1,5605,11,0,11,0.0
C1,9663,11,0,11,0.0
C3,989,11,0,11,0.0
C1,3167,11,0,11,0.0
C5,97674,11,0,11,0.0
C1,3470,11,0,11,0.0
C5,797,11,0,11,0.0
C8,1286,10,1,11,0.09091
C8,42,11,0,11,0.0
C1,4053,10,1,11,0.09091
C1,7887,11,0,11,0.0
C8,2683,11,0,11,0.0
C1,7114,11,0,11,0.0
C1,4023,11,0,11,0.0
C1,7330,12,0,12,0.0
C1,7945,12,0,12,0.0
C1,9983,12,0,12,0.0
C8,2395,12,0,12,0.0
C1,2410,12,0,12,0.0
C1,8602,11,1,12,0.08333
C1,6641,11,1,12,0.08333
C1,8816,11,1,12,0.08333
C1,7791,12,0,12,0.0
C3,1379,12,0,12,0.0
C1,8306,12,0,12,0.0
C1,997,8,4,12,0.33333
C1,3239,13,0,13,0.0
C1,1777,12,1,13,0.07692
C1,1534,13,0,13,0.0
C1,7968,13,0,13,0.0
C1,9770,13,0,13,0.0
C1,4014

In [ ]:
for i in range(0, len(df)):
    row = df.loc[i]
    label = row['Label']
    for j in range(1, 9):
        field = 'C{0}'.format(j)
        value = row[field]
        if label == 0:
            counts[field+','+str(value)][0] += 1
        else:
            counts[field+','+str(value)][1] += 1
        counts[field+','+str(value)][2] += 1

print('Field,Value,Neg,Pos,Total,Ratio')
for key, (neg, pos, total) in sorted(counts.items(), key=lambda x: x[1][2]):
    if total < 10:
        continue
    ratio = round(float(pos)/total, 5)
    print(key+','+str(neg)+','+str(pos)+','+str(total)+','+str(ratio))
    print(key+','+str(neg)+','+str(pos)+','+str(total)+','+str(ratio))

In [28]:
countList=[]
for i in range(0,8):
    countList.append(collections.defaultdict(lambda : [0, 0, 0]))
    
for i in range(0, 2000):
    row = df.loc[i]
    label = row['Label']
    for j in range(1, 9):
        field = 'C{0}'.format(j)
        value = row[field]
        if label == 0:
            countList[j-1][field+','+str(value)][0] += 1
        else:
            countList[j-1][field+','+str(value)][1] += 1
        countList[j-1][field+','+str(value)][2] += 1

print('Field,Value,Neg,Pos,Total,Ratio')
for i in range(1,9):
    for key, (neg, pos, total) in sorted(countList[i-1].items(), key=lambda x: x[1][2]):
        if total < 10:
            continue
        ratio = round(float(pos)/total, 5)
        print(key+','+str(neg)+','+str(pos)+','+str(total)+','+str(ratio))

Field,Value,Neg,Pos,Total,Ratio
C1,333,13,1,14,0.07143
C1,3904,15,0,15,0.0
C1,3411,18,0,18,0.0
C1,2485,19,1,20,0.05
C1,1600,21,0,21,0.0
C1,8671,26,0,26,0.0
C1,8491,30,0,30,0.0
C1,9399,36,0,36,0.0
C1,6621,44,1,45,0.02222
C1,2187,46,1,47,0.02128
C1,443,50,0,50,0.0
C1,2621,58,1,59,0.01695
C1,5165,61,0,61,0.0
C1,2338,67,0,67,0.0
C1,9034,69,0,69,0.0
C1,1651,70,0,70,0.0
C1,9460,71,0,71,0.0
C1,5825,79,2,81,0.02469
C1,4632,91,0,91,0.0
C1,7923,91,1,92,0.01087
C1,2830,91,1,92,0.01087
C1,3176,111,1,112,0.00893
C1,9655,115,0,115,0.0
C1,639,120,1,121,0.00826
C1,1188,130,1,131,0.00763
C1,1009,144,1,145,0.0069
C1,2591,251,0,251,0.0
C2,509660095530134768,209,3,212,0.01415
C2,5755694407684602296,608,2,610,0.00328
C2,8277336076276184272,1153,8,1161,0.00689
C3,2011,13,1,14,0.07143
C3,948,21,0,21,0.0
C3,123,79,2,81,0.02469
C3,1594,1842,10,1852,0.0054
C4,9,22,1,23,0.04348
C4,101,25,0,25,0.0
C4,99,79,2,81,0.02469
C4,50,1855,10,1865,0.00536
C6,2,32,0,32,0.0
C6,1,351,3,354,0.00847
C6,0,1604,10,1614,0.0062
C7,

In [27]:
countList=[]
for i in range(0,8):
    countList.append(collections.defaultdict(lambda : [0, 0, 0]))
    
len(countList)

8

In [2]:

print('Field,Value,Neg,Pos,Total,Ratio')
for key, (neg, pos, total) in sorted(counts.items(), key=lambda x: x[1][2]):
    if total < 10:
        continue
    ratio = round(float(pos)/total, 5)
    print(key+','+str(neg)+','+str(pos)+','+str(total)+','+str(ratio))


Field,Value,Neg,Pos,Total,Ratio
C1,5160,10,0,10,0.0
C1,6013,10,0,10,0.0
C3,5,9,1,10,0.1
C1,7310,10,0,10,0.0
C1,4224,10,0,10,0.0
C8,1130,9,1,10,0.1
C3,565,10,0,10,0.0
C8,1140,9,1,10,0.1
C1,4226,10,0,10,0.0
C1,4751,10,0,10,0.0
C3,1476,9,1,10,0.1
C8,2881,10,0,10,0.0
C3,1471,10,0,10,0.0
C1,9981,10,0,10,0.0
C4,23,10,0,10,0.0
C8,2542,10,0,10,0.0
C1,4773,9,1,10,0.1
C1,4457,10,0,10,0.0
C8,1871,10,0,10,0.0
C1,1180,9,1,10,0.1
C8,3935,10,0,10,0.0
C1,9860,10,0,10,0.0
C8,1211,11,0,11,0.0
C8,65,10,1,11,0.09091
C3,1297,11,0,11,0.0
C5,77016,11,0,11,0.0
C3,1596,11,0,11,0.0
C1,8441,11,0,11,0.0
C1,4628,11,0,11,0.0
C8,3618,11,0,11,0.0
C3,1784,11,0,11,0.0
C1,5979,11,0,11,0.0
C1,4384,11,0,11,0.0
C1,9225,11,0,11,0.0
C8,714,11,0,11,0.0
C8,1773,11,0,11,0.0
C8,2886,11,0,11,0.0
C8,2053,11,0,11,0.0
C1,7710,11,0,11,0.0
C8,637,11,0,11,0.0
C4,48,12,0,12,0.0
C1,4541,11,1,12,0.08333
C1,6156,12,0,12,0.0
C1,5520,12,0,12,0.0
C1,4000,13,0,13,0.0
C1,2404,13,0,13,0.0
C1,8201,12,1,13,0.07692
C1,1799,13,0,13,0.0
C1,4865,13,0,

In [38]:
target_cat_feats=['C1-8268', 'C1-3295', 'C1-5927', 'C1-2279', 'C1-3026', 'C1-6272', 'C1-5985', 'C1-3040', 
                  'C2-8277336076276184272', 'C2-5755694407684602296', 'C2-509660095530134768', 'C2-5799347067982556520', 'C2-7258015885215914736', 
                  'C3-1755', 'C3-1594', 'C3-1144', 'C3-92', 'C3-160', 'C3-1533', 'C3-427', 'C3-1049', 'C3-1492', 'C3-1836', 'C3-1322', 'C3-844', 'C3-1584', 'C3-1609', 'C3-834', 'C3-261', 'C3-413',
                  'C4-101', 'C4-99', 'C4-50', 'C4-78', 'C4-1', 'C4-38', 'C4-14', 'C4-60', 'C4-66', 'C4-91', 'C4-25', 'C4-115', 'C4-48', 'C4-65', 'C4-90',
                  'C5-154495', 'C5-194672', 'C5-90249', 'C5-129548', 'C5-153329', 'C5-191015', 'C5-13650', 'C5-135894', 'C5-90445',
                  'C6-0', 'C6-1',
                  'C7-2002', 'C7-2005',
                  'C8-2802', 'C8-788', 'C8-3841', 'C8-829', 'C8-1750', 'C8-1791', 'C8-2477', 'C8-2124', 'C8-2549', 'C8-1856', 'C8-1071']

In [41]:
    target_cat_feats=cat_set
    for row in csv.DictReader(open('te_8k.csv')):
        feats = []
        for j in range(1, 14):
            val = row['I{0}'.format(j)]
            if val == '':
                val = -10
            feats.append('{0}'.format(val))
        #print(row['Label'] + ' ' + ' '.join(feats)+'\n')
        #print('--------------------------')
        #f_d.write(row['Label'] + ' ' + ' '.join(feats) + '\n')
        
        cat_feats = set()
        for j in range(1, 9):
            field = 'C{0}'.format(j)
            if j == 2:
                for i in row[field].split(';'):
                    key = 'C2-' + i
                    cat_feats.add(key)
            else:
                key = field + '-' + row[field]
                cat_feats.add(key)
        print(list(cat_feats))
        print('+++++++++++++++++++++++++++')
        feats = []
        for j, feat in enumerate(target_cat_feats, start=1):
            if feat in cat_feats:
                feats.append(str(j))
        print(row['Label'] + ' ' + ' '.join(feats)+'\n')
        print('--------------------------')
        #f_s.write(row['Label'] + ' ' + ' '.join(feats) + '\n')


['C2-7908382889764677758', 'C7-2005', 'C1-7495', 'C4-101', 'C8-42', 'C6-1', 'C5-39217', 'C2-509660095530134768', 'C3-427']
+++++++++++++++++++++++++++
0 11 20 31 56 58

--------------------------
['C5-95174', 'C2-7908382889764677758', 'C7-2005', 'C1-7495', 'C4-101', 'C6-0', 'C8-42', 'C2-509660095530134768', 'C3-427']
+++++++++++++++++++++++++++
0 11 20 31 55 58

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C5-93434', 'C4-101', 'C2-5755694407684602296', 'C6-1', 'C8-42', 'C1-5520', 'C3-427']
+++++++++++++++++++++++++++
0 10 20 31 56 58

--------------------------
['C2-7908382889764677758', 'C5-46317', 'C7-2002', 'C4-101', 'C1-2717', 'C6-0', 'C8-42', 'C2-509660095530134768', 'C3-427']
+++++++++++++++++++++++++++
0 11 20 31 55 57

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C4-101', 'C1-2717', 'C6-0', 'C8-42', 'C2-509660095530134768', 'C5-113877', 'C3-427']
+++++++++++++++++++++++++++
0 11 20 31 55 58

--------------------------
['C2-7908382889764


--------------------------
['C2-7908382889764677758', 'C7-2005', 'C4-50', 'C2-8277336076276184272', 'C6-1', 'C8-1398', 'C5-152149', 'C3-1755', 'C1-9171']
+++++++++++++++++++++++++++
0 9 14 33 56 58

--------------------------
['C5-186792', 'C2-7908382889764677758', 'C4-50', 'C7-2002', 'C6-0', 'C8-1398', 'C2-8277336076276184272', 'C3-1755', 'C1-9171']
+++++++++++++++++++++++++++
0 9 14 33 55 57

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C2-7258015885215914736', 'C5-20168', 'C6-0', 'C3-1755', 'C4-26', 'C8-3571', 'C1-3287']
+++++++++++++++++++++++++++
0 13 14 55 58

--------------------------
['C2-7908382889764677758', 'C2-7258015885215914736', 'C7-2002', 'C5-172277', 'C6-0', 'C3-1755', 'C4-26', 'C8-3571', 'C1-3287']
+++++++++++++++++++++++++++
0 13 14 55 57

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C2-7258015885215914736', 'C6-0', 'C5-130355', 'C3-1755', 'C4-26', 'C8-3571', 'C1-3287']
+++++++++++++++++++++++++++
0 13 14 55 58

-----------

+++++++++++++++++++++++++++
0 9 31 55 58

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C4-101', 'C5-47718', 'C3-1432', 'C8-3489', 'C6-0', 'C2-8277336076276184272', 'C1-8230']
+++++++++++++++++++++++++++
0 9 31 55 58

--------------------------
['C2-7908382889764677758', 'C7-2004', 'C3-1432', 'C4-101', 'C8-3489', 'C1-8238', 'C6-0', 'C5-31277', 'C2-509660095530134768']
+++++++++++++++++++++++++++
0 11 31 55

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C3-1432', 'C4-101', 'C8-3489', 'C5-45536', 'C2-5755694407684602296', 'C1-2333', 'C6-0']
+++++++++++++++++++++++++++
0 10 31 55 58

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C3-1432', 'C5-52217', 'C4-101', 'C8-3489', 'C1-8238', 'C6-0', 'C2-509660095530134768']
+++++++++++++++++++++++++++
0 11 31 55 58

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C4-101', 'C3-1432', 'C8-3489', 'C6-0', 'C2-8277336076276184272', 'C5-126485', 'C1-8230']
+++++++++++++++++++++

+++++++++++++++++++++++++++
0 11 31 55 58

--------------------------
['C2-7908382889764677758', 'C7-2002', 'C4-101', 'C3-1432', 'C8-3489', 'C6-1', 'C5-184648', 'C2-8277336076276184272', 'C1-8230']
+++++++++++++++++++++++++++
0 9 31 56 57

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C4-101', 'C3-1432', 'C5-186390', 'C8-3489', 'C6-0', 'C2-8277336076276184272', 'C1-8230']
+++++++++++++++++++++++++++
0 9 31 55 58

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C5-167997', 'C4-101', 'C3-1432', 'C8-3489', 'C6-0', 'C2-8277336076276184272', 'C1-8230']
+++++++++++++++++++++++++++
0 9 31 55 58

--------------------------
['C2-7908382889764677758', 'C5-131248', 'C7-2005', 'C4-101', 'C3-1432', 'C8-3489', 'C6-1', 'C2-8277336076276184272', 'C1-8230']
+++++++++++++++++++++++++++
0 9 31 56 58

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C4-101', 'C3-1432', 'C8-3489', 'C6-0', 'C2-8277336076276184272', 'C5-22138', 'C1-4566']
+++++++++++++++

+++++++++++++++++++++++++++
0 12 14 31 55

--------------------------
['C5-23316', 'C1-2323', 'C2-7908382889764677758', 'C4-50', 'C7-2005', 'C8-1183', 'C2-5755694407684602296', 'C6-1', 'C3-527']
+++++++++++++++++++++++++++
0 10 33 56 58

--------------------------
['C2-7908382889764677758', 'C1-2075', 'C4-50', 'C7-2005', 'C8-1183', 'C2-5755694407684602296', 'C6-1', 'C3-527', 'C5-135301']
+++++++++++++++++++++++++++
0 10 33 56 58

--------------------------
['C2-7908382889764677758', 'C1-2075', 'C4-50', 'C5-161561', 'C7-2005', 'C8-1183', 'C2-5755694407684602296', 'C3-527', 'C6-0']
+++++++++++++++++++++++++++
0 10 33 55 58

--------------------------
['C2-7908382889764677758', 'C1-2075', 'C4-50', 'C7-2005', 'C8-1183', 'C5-70595', 'C2-5755694407684602296', 'C6-1', 'C3-527']
+++++++++++++++++++++++++++
0 10 33 56 58

--------------------------
['C1-2323', 'C2-7908382889764677758', 'C4-50', 'C7-2005', 'C8-1183', 'C5-6256', 'C2-5755694407684602296', 'C3-527', 'C6-0']
++++++++++++++++++++++++

...

['C2-7908382889764677758', 'C3-526', 'C7-2005', 'C4-101', 'C5-29713', 'C1-8773', 'C6-0', 'C2-8277336076276184272', 'C8-2774']
+++++++++++++++++++++++++++
0 9 31 55 58

--------------------------
['C2-7908382889764677758', 'C3-526', 'C7-2005', 'C4-101', 'C6-1', 'C5-74886', 'C8-2774', 'C2-509660095530134768', 'C1-9111']
+++++++++++++++++++++++++++
0 11 31 56 58

--------------------------
['C2-7908382889764677758', 'C3-526', 'C7-2005', 'C4-101', 'C5-43991', 'C6-0', 'C8-2774', 'C2-509660095530134768', 'C1-9111']
+++++++++++++++++++++++++++
0 11 31 55 58

--------------------------
['C2-7908382889764677758', 'C3-526', 'C5-78376', 'C7-2002', 'C4-101', 'C6-0', 'C2-8277336076276184272', 'C8-2774', 'C1-8985']
+++++++++++++++++++++++++++
0 9 31 55 57

--------------------------
['C2-7908382889764677758', 'C3-526', 'C7-2002', 'C4-101', 'C5-101428', 'C6-0', 'C2-8277336076276184272', 'C8-2774', 'C1-8985']
+++++++++++++++++++++++++++
0 9 31 55 57

--------------------------
['C2-7908382889764677758

['C2-7908382889764677758', 'C5-79067', 'C7-2005', 'C4-101', 'C8-1863', 'C1-7693', 'C3-1602', 'C6-0', 'C2-8277336076276184272']
+++++++++++++++++++++++++++
0 9 31 55 58

--------------------------
['C2-7908382889764677758', 'C7-2002', 'C4-101', 'C8-1863', 'C6-1', 'C1-7693', 'C3-1602', 'C5-108605', 'C2-8277336076276184272']
+++++++++++++++++++++++++++
0 9 31 56 57

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C4-101', 'C8-1863', 'C1-7693', 'C3-1602', 'C5-145085', 'C6-0', 'C2-8277336076276184272']
+++++++++++++++++++++++++++
0 9 31 55 58

--------------------------
['C2-7908382889764677758', 'C7-2002', 'C4-101', 'C8-1863', 'C1-7693', 'C3-1602', 'C6-0', 'C2-8277336076276184272', 'C5-53994']
+++++++++++++++++++++++++++
0 9 31 55 57

--------------------------
['C5-170637', 'C2-7908382889764677758', 'C7-2005', 'C4-101', 'C8-1863', 'C3-1602', 'C6-0', 'C2-8277336076276184272', 'C1-9085']
+++++++++++++++++++++++++++
0 9 31 55 58

--------------------------
['C2-790838288976

--------------------------
['C4-90', 'C2-7908382889764677758', 'C7-2005', 'C3-622', 'C1-9817', 'C2-5755694407684602296', 'C5-560', 'C6-0', 'C8-1424']
+++++++++++++++++++++++++++
0 10 45 55 58

--------------------------
['C4-90', 'C2-7908382889764677758', 'C7-2002', 'C3-622', 'C1-9817', 'C2-5755694407684602296', 'C6-0', 'C8-1424', 'C5-80071']
+++++++++++++++++++++++++++
0 10 45 55 57

--------------------------
['C4-90', 'C2-7908382889764677758', 'C7-2002', 'C3-622', 'C1-9817', 'C5-109228', 'C2-5755694407684602296', 'C6-0', 'C8-1424']
+++++++++++++++++++++++++++
0 10 45 55 57

--------------------------
['C4-90', 'C2-7908382889764677758', 'C7-2002', 'C3-622', 'C1-9817', 'C2-5755694407684602296', 'C6-0', 'C8-1424', 'C5-42097']
+++++++++++++++++++++++++++
1 10 45 55 57

--------------------------
['C4-90', 'C2-7908382889764677758', 'C7-2002', 'C5-163874', 'C3-622', 'C1-9817', 'C2-5755694407684602296', 'C6-0', 'C8-1424']
+++++++++++++++++++++++++++
0 10 45 55 57

-------------------------

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C4-101', 'C1-5582', 'C5-53561', 'C3-1674', 'C6-0', 'C8-3456', 'C2-8277336076276184272']
+++++++++++++++++++++++++++
0 9 31 55 58

--------------------------
['C2-7908382889764677758', 'C7-2002', 'C5-113822', 'C4-101', 'C3-1674', 'C2-5755694407684602296', 'C1-1362', 'C6-0', 'C8-3456']
+++++++++++++++++++++++++++
0 10 31 55 57

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C4-101', 'C3-1674', 'C2-5755694407684602296', 'C1-1362', 'C6-0', 'C8-3456', 'C5-170267']
+++++++++++++++++++++++++++
0 10 31 55 58

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C4-101', 'C3-1674', 'C2-5755694407684602296', 'C1-1362', 'C5-66775', 'C6-0', 'C8-3456']
+++++++++++++++++++++++++++
0 10 31 55 58

--------------------------
['C5-59024', 'C2-7908382889764677758', 'C7-2005', 'C4-101', 'C1-5582', 'C3-1674', 'C6-0', 'C8-3456', 'C2-8277336076276184272']
+++++++++++++++++++++++++++
0 9 31 55 58

---------------


--------------------------
['C2-7908382889764677758', 'C7-2005', 'C2-7258015885215914736', 'C4-50', 'C1-770', 'C8-3892', 'C6-0', 'C3-1755', 'C5-46738']
+++++++++++++++++++++++++++
0 13 14 33 55 58

--------------------------
['C2-7908382889764677758', 'C2-7258015885215914736', 'C4-50', 'C7-2002', 'C1-770', 'C8-3892', 'C6-0', 'C5-26344', 'C3-1755']
+++++++++++++++++++++++++++
0 13 14 33 55 57

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C2-7258015885215914736', 'C4-50', 'C1-770', 'C8-3892', 'C5-21142', 'C6-0', 'C3-1755']
+++++++++++++++++++++++++++
0 13 14 33 55 58

--------------------------
['C2-7908382889764677758', 'C7-2005', 'C2-7258015885215914736', 'C4-50', 'C1-770', 'C8-3892', 'C6-0', 'C3-1755', 'C5-162545']
+++++++++++++++++++++++++++
0 13 14 33 55 58

--------------------------
['C2-7908382889764677758', 'C2-7258015885215914736', 'C4-50', 'C7-2004', 'C1-770', 'C8-3892', 'C5-80776', 'C6-0', 'C3-1755']
+++++++++++++++++++++++++++
0 13 14 33 55

-----------

In [28]:
df1 = pd.DataFrame(pd.read_csv('te_8k.csv', header=0, delim_whitespace=True))
df1.columns

Index(['Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8'], dtype='object')

In [29]:
df1.shape

(7999, 1)

In [17]:
df2.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
